<h1>Welcome to the Reflection Simulation!</h1>

In [ ]:
# THESE ARE JUST LIBRARIES
from vpython import *
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore', 'The iteration is not making good progress')

# THIS SETS UP THE RENDERING SPACE
scene = canvas(background=color.white)
scene.userzoom = False
scene.userspin = False
base = box(size = vector(35,0.5,0.9), pos = vector(0,-4,0))

# STORES BOUNDARY CONDITIONS
coords = []

# THIS CREATES ENDPOINTS WHEN THE USER CLICKS
def showSphere(evt):
    loc = evt.pos
    point = sphere(pos=loc, color=color.cyan)
    coords.append(point.pos)

scene.bind('click', showSphere)

while len(coords)<2:
    rate(5)
    
scene.unbind('click', showSphere)
    
incRef = vec(0,-1,0)

# THESE GRAPHS VISUALIZE INCIDENCE ANGLE, REFLECTED ANGLE, & ACTION
tgraph=graph(title = 'Trajectory', xtitle="Angle [Deg]", ytitle="Light Time [s]", fast=False, width=600, height=400)
f1=gcurve(graph = tgraph, color=color.blue, label = 'Incident', legend = True, markers = True, marker_color = color.blue)
f2=gcurve(graph = tgraph, color=color.green, label = 'Reflected', legend = True, markers = True, marker_color = color.green)
f3=gdots(graph = tgraph, label = 'True Incident', legend = True, radius = 6, color = color.red)
f4=gdots(graph = tgraph, label = 'True Reflected', legend = True, radius = 6,color = color.red)
ngraph=graph(title = 'Path of Least Action', xtitle="Incident : Reflected", ytitle="Action", fast=False, width=600, height=400)
f5=gcurve(graph = ngraph, color=color.red, label = 'Action', legend = True, markers = True, marker_color = color.red)

t = 0
dt = 0.01

# CALCULATING ASSUMING 500NM GREEN LIGHT; E/c^2 = m = h/(lambda * c)
m = 4.41e-36
KE = 0.5 * m * (3*(10**8))**2

# PARAMETERS FOR OPTIMIZATION FUNCTION
a = coords[1].y
b = coords[0].y
L = coords[0].x-coords[1].x

# THIS IS THE DIFFERENTIAL EQUATION WHOSE SOLUTION DETERMINES THE ANGLE OF INCIDENCE & REFLECTION
def f(z):
    return ( sqrt(a**2 + z**2) + sqrt(b**2 + (L-z)**2) ) / 3

# ARBRITRARY GUESS FOR SCIPY TO SOLVE DIFF. EQ.
x0 = 1.0
x = fsolve(f,x0)

# HITPOINT LOCATION DETERMINED BY SOLUTION TO OPTIMIZATION FUNCTION
hitpoint = vec( coords[1].x+x , -3.6, 0)
v2 = vec( ( coords[0].x+coords[1].x ) / 2 , -3.6, 0)

# ENABLES USER TO CREATE BOUNDARY CONDITIONS LEFT-TO-RIGHT & VICE-VERSA
if coords[0].x<coords[1].x:
    v1 = coords[0]
    v3 = coords[1]
else:
    v1 = coords[1]
    v3 = coords[0]
    
# RUNS ONCE TWO ENDPOINTS ARE CREATED
if len(coords) == 2:
    
    # ALL LIGHT RAYS BETWEEN THE ENDPOINTS ARE DRAWN, HENCE THE RANGE OF THE FOR LOOP
    for i in range(int(v1.x),int(v3.x)):
    
        rate(10)
        
        # ITERATES AND DRAWS ALL POSSIBLE LIGHT RAYS
        v2.x = i
        c = curve(pos = [v1,v2], radius = 0.1)
        c.append(pos = v3, color = color.yellow)
        
        # USES diff_angle() METHOD TO FIND INCIDENT & REFLECTED ANGLES
        incident = degrees(diff_angle(incRef,v2-v1))
        reflected = degrees(pi - diff_angle(incRef,v3-v2))
        
        #SCALE FACTOR FOR TIME
        t = ( mag(v2-v1) + mag(v3-v2) ) / (3)
        
        # PHOTON HAS NO POTENTIAL ENERGY
        action = KE * (t * 3)
        
        # PLOTS PARAMETERS
        f1.plot(incident,t)
        f2.plot(reflected,t)
        f5.plot(incident/reflected, action)

# THIS DISPLAYS THE TRUE LIGHT PATH
d = curve(pos = [v1,hitpoint], radius = 0.2, color = color.red)
d.append(pos = v3, color = color.red)

# CALCULATES PARAMETERS FOR LEAST-TIME PATH
true_incident = degrees(diff_angle(incRef,hitpoint-v1))
true_reflected = degrees(pi - diff_angle(incRef,v3-hitpoint))
true_time = ( mag(hitpoint-v1) + mag(v3-hitpoint) ) / (3)

# PLOTS PARAMETERS FOR LEAST-TIME PATH
f3.plot(true_incident,true_time)
f4.plot(true_reflected,true_time)

# INFORMS USER OF TRUE PATH'S PARAMETRS
print("True (Incident, Reflected): ("+ str(round(true_incident)) + ", "+str(round(true_reflected))+")")

<h3><mark style = "background: #ed462f; color: white;">Please wait. The simulation is loading.</mark> </h3>
    
You will see a gray metal bar. Click above it to create the start point for the light ray. Click somewhere else to create the end point. 